<a href="https://colab.research.google.com/github/mehenika/Log_level_identification/blob/main/method_extractor_real.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install javalang

In [ ]:
import os
import re
import csv
import javalang
from google.colab import drive

def get_method_start_end(method_node, tree):
    startpos  = None
    endpos    = None
    startline = None
    endline   = None
    for path, node in tree:
        if startpos is not None and method_node not in path and endpos is None:
            endpos = node.position
            endline = node.position.line if node.position is not None else None
        if startpos is None and node == method_node:
            startpos = node.position
            startline = node.position.line if node.position is not None else None

        if startpos is not None and endpos is not None:
          break

    return startpos, endpos, startline, endline

def get_method_text(startpos, endpos, startline, endline, last_endline_index, codelines):
    if startpos is None:
        return "", None, None, None
    else:
        startline_index = startline - 1
        endline_index = endline - 1 if endpos is not None else None

        # 1. check for and fetch annotations
        if last_endline_index is not None:
            for line in codelines[(last_endline_index + 1):(startline_index)]:
                if "@" in line:
                    startline_index = startline_index - 1
        meth_text = "<ST>".join(codelines[startline_index:endline_index])
        meth_text = meth_text[:meth_text.rfind("}") + 1]

        # 2. remove trailing rbrace for last methods & any external content/comments
        # if endpos is None and
        if not abs(meth_text.count("}") - meth_text.count("{")) == 0:
            # imbalanced braces
            brace_diff = abs(meth_text.count("}") - meth_text.count("{"))

            for _ in range(brace_diff):
                meth_text  = meth_text[:meth_text.rfind("}")]
                meth_text  = meth_text[:meth_text.rfind("}") + 1]

        meth_lines = meth_text.split("<ST>")
        meth_text  = "".join(meth_lines)
        last_endline_index = startline_index + (len(meth_lines) - 1)

        return meth_text, (startline_index + 1), (last_endline_index + 1), last_endline_index

def extract_methods(file_content, codelines):
    tree = javalang.parse.parse(file_content)
    methods = []
    lex = None
    for _, node in tree.filter(javalang.tree.MethodDeclaration):
        method_name = node.name
        startpos, endpos, startline, endline = get_method_start_end(node, tree)
        method_content, startline, endline, lex = get_method_text(startpos, endpos, startline, endline, lex, codelines)
        methods.append((method_name, method_content.strip()))

    return methods

def process_java_files(folder_path):
    all_methods = []
    for java_file in os.listdir(folder_path):
        if java_file.endswith('.java'):
            file_path = os.path.join(folder_path, java_file)
            codelines = None
            with open(file_path, 'r') as r:
                codelines = r.readlines()
            with open(file_path, 'r', encoding='utf-8') as file:
                file_content = file.read()
                print(file_path)
                methods = extract_methods(file_content, codelines)
                all_methods.extend(methods)
    return all_methods

def save_to_csv(output_file, methods):
    with open(output_file, 'w', newline='', encoding='utf-8') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['Method Name', 'Method Content'])
        csv_writer.writerows(methods)

def main(folder_path, output_file):
    drive.mount('/content/drive')
    methods = process_java_files(folder_path)
    save_to_csv(output_file, methods)

if __name__ == "__main__":
    input_folder = '/content/drive/MyDrive/Thesis_Brock/JavaFilesDataset/jmeterf2'
    output_csv = '/content/drive/MyDrive/Thesis_Brock/Output_Folder/jmeter_methods.csv'
    main(input_folder, output_csv)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Thesis_Brock/JavaFilesDataset/jmeterf2/JSONPostProcessor.java
/content/drive/MyDrive/Thesis_Brock/JavaFilesDataset/jmeterf2/JSR223Assertion.java
/content/drive/MyDrive/Thesis_Brock/JavaFilesDataset/jmeterf2/MainFrame.java
/content/drive/MyDrive/Thesis_Brock/JavaFilesDataset/jmeterf2/JMESPathAssertion.java
/content/drive/MyDrive/Thesis_Brock/JavaFilesDataset/jmeterf2/AbstractSampleConsumer.java
/content/drive/MyDrive/Thesis_Brock/JavaFilesDataset/jmeterf2/XPathExtractor.java
/content/drive/MyDrive/Thesis_Brock/JavaFilesDataset/jmeterf2/JsonExporter.java
/content/drive/MyDrive/Thesis_Brock/JavaFilesDataset/jmeterf2/SelectTemplatesDialog.java
/content/drive/MyDrive/Thesis_Brock/JavaFilesDataset/jmeterf2/HttpMetricsSender.java
/content/drive/MyDrive/Thesis_Brock/JavaFilesDataset/jmeterf2/SplashScreen.java
/content/drive/MyDrive/Thesis_Brock

KeyboardInterrupt: ignored

In [ ]:
import os
import re
import csv
import javalang
from google.colab import drive
import shutil  # Import shutil for file operations
from javalang.parser import JavaSyntaxError

def get_method_start_end(method_node, tree):
    startpos  = None
    endpos    = None
    startline = None
    endline   = None
    for path, node in tree:
        if startpos is not None and method_node not in path and endpos is None:
            endpos = node.position
            endline = node.position.line if node.position is not None else None
        if startpos is None and node == method_node:
            startpos = node.position
            startline = node.position.line if node.position is not None else None

        if startpos is not None and endpos is not None:
          break

    return startpos, endpos, startline, endline

def get_method_text(startpos, endpos, startline, endline, last_endline_index, codelines):
    if startpos is None:
        return "", None, None, None
    else:
        startline_index = startline - 1
        endline_index = endline - 1 if endpos is not None else None

        # 1. check for and fetch annotations
        if last_endline_index is not None:
            for line in codelines[(last_endline_index + 1):(startline_index)]:
                if "@" in line:
                    startline_index = startline_index - 1
        meth_text = "<ST>".join(codelines[startline_index:endline_index])
        meth_text = meth_text[:meth_text.rfind("}") + 1]

        # 2. remove trailing rbrace for last methods & any external content/comments
        # if endpos is None and
        if not abs(meth_text.count("}") - meth_text.count("{")) == 0:
            # imbalanced braces
            brace_diff = abs(meth_text.count("}") - meth_text.count("{"))

            for _ in range(brace_diff):
                meth_text  = meth_text[:meth_text.rfind("}")]
                meth_text  = meth_text[:meth_text.rfind("}") + 1]

        meth_lines = meth_text.split("<ST>")
        meth_text  = "".join(meth_lines)
        last_endline_index = startline_index + (len(meth_lines) - 1)

        return meth_text, (startline_index + 1), (last_endline_index + 1), last_endline_index

def extract_methods(file_content, codelines):
    try:
        tree = javalang.parse.parse(file_content)
        methods = []
        lex = None
        for _, node in tree.filter(javalang.tree.MethodDeclaration):
            method_name = node.name
            startpos, endpos, startline, endline = get_method_start_end(node, tree)
            method_content, startline, endline, lex = get_method_text(startpos, endpos, startline, endline, lex, codelines)
            methods.append((method_name, method_content.strip()))
        return methods, None  # No error
    except JavaSyntaxError as e:
        return None, str(e)  # Return the error message
    except Exception as e:
        return None, str(e)  # Return the error message for any other exception

def process_java_files(input_folder, output_folder, error_folder):
    all_methods = []
    error_files = []
    for java_file in os.listdir(input_folder):
        if java_file.endswith('.java'):
            file_path = os.path.join(input_folder, java_file)
            codelines = None
            with open(file_path, 'r') as r:
                codelines = r.readlines()
            with open(file_path, 'r', encoding='utf-8') as file:
                file_content = file.read()
                print(file_path)
                methods, error_message = extract_methods(file_content, codelines)
                if methods is not None:
                    all_methods.extend(methods)
                else:
                    print(f"Error in file '{java_file}': {error_message}")
                    error_files.append(java_file)
                    # Move the error file to the error folder
                    shutil.move(file_path, os.path.join(error_folder, java_file))

    return all_methods

def save_to_csv(output_file, methods):
    with open(output_file, 'w', newline='', encoding='utf-8') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['Method Name', 'Method Content'])
        csv_writer.writerows(methods)

def main(input_folder, output_csv, error_folder):
    drive.mount('/content/drive')
    methods = process_java_files(input_folder, output_csv, error_folder)
    save_to_csv(output_csv, methods)

if __name__ == "__main__":
    input_folder = '/content/drive/MyDrive/Thesis_Brock/JavaFilesDataset/hbasef2'
    output_csv = '/content/drive/MyDrive/Thesis_Brock/Output_Folder/hbase_methods.csv'
    error_folder = '/content/drive/MyDrive/Thesis_Brock/JavaFilesDataset/troubled_files/hbase'
    main(input_folder, output_csv, error_folder)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Thesis_Brock/JavaFilesDataset/hbasef2/RestoreTool.java
/content/drive/MyDrive/Thesis_Brock/JavaFilesDataset/hbasef2/SimpleRequestController.java
/content/drive/MyDrive/Thesis_Brock/JavaFilesDataset/hbasef2/CompatibilitySingletonFactory.java
/content/drive/MyDrive/Thesis_Brock/JavaFilesDataset/hbasef2/CatalogReplicaLoadBalanceSimpleSelector.java
/content/drive/MyDrive/Thesis_Brock/JavaFilesDataset/hbasef2/AsyncRegionLocatorHelper.java
/content/drive/MyDrive/Thesis_Brock/JavaFilesDataset/hbasef2/BuiltInProviderSelector.java
/content/drive/MyDrive/Thesis_Brock/JavaFilesDataset/hbasef2/HBaseMetrics2HadoopMetricsAdapter.java
/content/drive/MyDrive/Thesis_Brock/JavaFilesDataset/hbasef2/ReadOnlyZKClient.java
/content/drive/MyDrive/Thesis_Brock/JavaFilesDataset/hbasef2/RefreshHFilesClient.java
/content/drive/MyDrive/Thesis_Brock/JavaFilesDatase